Model_0.1.0
Consultor Agent specilized in Medical Devices Regulations



In [ ]:
import requests
import PyPDF2
import re
from PyPDF2 import PdfReader
import os
import requests
import chromadb
import google.generativeai as genai
import dotenv
from dotenv import load_dotenv
import chromadb.utils.embedding_functions as embedding_functions

In [9]:
## Configure Gemini and API KEY

key_path = r"C:\Users\albfr\Desktop\BeQu\API Key\gemini_api_key1.env"
load_dotenv(key_path)

# Accede a la API key
api_key_1 = os.getenv("gemini_api_key1")

In [10]:
## Checking Gemini Configuration API

genai.configure(api_key= os.getenv("gemini_api_key1"))

for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/learnlm-1.5-pro-experimental


In [12]:
### Function: Exttraction text from the .pdf

file_path=r"C:\Users\albfr\Desktop\BeQu\data\medicalstandard.pdf"

def load_pdf(file_path):
  pdf_reader = PdfReader(file_path)
  text = ""
  
  for page in pdf_reader.pages:
    text += page.extract_text()
  return text


### Checking the .pdf


pdf_text = load_pdf(file_path)

# pdf_text
pdf_text[:1500]

len(pdf_text)



119413

In [13]:
# Function to Split Text into smallers Chuncks

## Setup Max Chunck Lenght and the Chunck Overlap


def pdf_text_splitter(text, max_length=1000, chunk_overlap=0):
  text = re.sub(r'\.\.\.', '', text) # Remove ellipsis (...)
  chunks = []
  start = 0 
  text_length = len(text) # calculate the whole text lenght
  while start < text_length: # keep moving till the end of the full text
    end = start + max_length
    if end < text_length : # When we are not at the end of text
      end = text.rfind(' ', start, end) + 1 # For not cutting words at the middle. Ending chunck between spaces

      if end <= start: # When there is no space, split at the max length
        end = start + max_length

    chunk = text[start:end].strip() # Take text from Stat till End, and remove spaces in between

    if chunk:
      chunks.append(chunk) # When finding a Chucnk, adding to chuncks = []

    start = end - chunk_overlap  # moving start position forward minus any overlaps

    if start >= text_length: # When reaching out the end of the text
      break
  return chunks

In [14]:
chunks = pdf_text_splitter(pdf_text, max_length=2000, chunk_overlap=200)

# Checking Chunks


len(chunks)

chunks[0]

'© ISO 2016Medical devices  — Quality \nmanagement systems — \nRequirements for regulatory purposes\nDispositifs médicaux  — Systèmes de management de la qualité — \nExigences à des fins réglementairesINTERNATIONAL \nSTANDARDISO\n13485\nThird edition\n2016-03-01\nReference number\nISO 13485:2016(E)\n ISO 13485:2016(E)\n \nii © ISO 2016 – All rights reservedCOPYRIGHT PROTECTED DOCUMENT\n©  ISO 2016, Published in Switzerland\nAll rights reserved. Unless otherwise specified, no part of this publication may be reproduced or utilized otherwise in any form \nor by any means, electronic or mechanical, including photocopying, or posting on the internet or an intranet, without prior \nwritten permission. Permission can be requested from either ISO at the address below or ISO’s member body in the country of \nthe requester.\nISO copyright office\nCh. de Blandonnet 8 • CP 401\nCH-1214 Vernier, Geneva, Switzerland\nTel. +41 22 749 01 11\nFax +41 22 749 09 47\ncopyright@iso.org\nwww.iso.org\n ISO 1

In [15]:
## Embedding Chucnks in ChromaDB

google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key= os.getenv("gemini_api_key1"))
client = chromadb.PersistentClient(path="embeddings/gemini")


collection = client.get_or_create_collection(name="pdf_rag", embedding_function=google_ef)



for i, d in enumerate(chunks):
  collection.add(documents=[d], ids=[str(i)])






In [16]:
collection.count()

60

In [17]:
# Function for Context and Recover Context

def recover_context(context):
  recovered_context = ""
  for item in context:
    recovered_context += item + "\n\n"  ## To aggregate the results in one big answer
  return recovered_context

def text_relevant_in_context(query, db, n_results=3):
  results = db.query(query_texts=[query], n_results=n_results)
  recovered_context = recover_context(results['documents'][0])
  return recovered_context

In [39]:
context = text_relevant_in_context("How to develop a design validation for medical devices", collection )


# len(context)

In [34]:
# To verify how the context looks

print(context)

ication or intended use.
The organization shall document validation plans that include methods, acceptance criteria and, as 
appropriate, statistical techniques with rationale for sample size. 
© ISO 2016 – All rights reserved  15
 ISO 13485:2016(E)
Design validation shall be conducted on representative product. Representative product includes 
initial production units, batches or their equivalents. The rationale for the choice of product used for 
validation shall be recorded (see 4.2.5 ).
As part of design and development validation, the organization shall perform clinical evaluations or 
performance evaluations of the medical device in accordance with applicable regulatory requirements. 
A medical device used for clinical evaluation or performance evaluation is not considered to be released 
for use to the customer.
If the intended use requires that the medical device be connected to, or have an interface with, other 
medical device(s), validation shall include confirmation that the

In [35]:
# Prompt Designing 

def give_a_prompt_to_the_agent(query, context):
  prompt = f"""
  You are a consultor agent that answers questions using the text from the context below.
  The question and the context is going to be shared with you, so you can answer the question based on the context.
  If the context does not have enough information to answer the question properly, inform the user about the abscene of relevant  context as an answer

  Question : {query}
  \n
  Context : {context}
  \n
  Answer :
  """
  return prompt

In [36]:
# Function to define the answer based on the prompt

def generate_answer(prompt):
  model = genai.GenerativeModel('gemini-1.5-flash-latest')
  result = model.generate_content(prompt)
  return result

In [40]:
# Prompt


prompt = give_a_prompt_to_the_agent(query="How to develop a design validation for medical devices", context=context)

print(prompt)


  You are a consultor agent that answers questions using the text from the context below.
  The question and the context is going to be shared with you, so you can answer the question based on the context.
  If the context does not have enough information to answer the question properly, inform the user about the abscene of relevant  context as an answer

  Question : How to develop a design validation for medical devices
  

  Context : ication or intended use.
The organization shall document validation plans that include methods, acceptance criteria and, as 
appropriate, statistical techniques with rationale for sample size. 
© ISO 2016 – All rights reserved  15
 ISO 13485:2016(E)
Design validation shall be conducted on representative product. Representative product includes 
initial production units, batches or their equivalents. The rationale for the choice of product used for 
validation shall be recorded (see 4.2.5 ).
As part of design and development validation, the organizatio

In [38]:
answer = generate_answer(prompt)
print(answer.text)


Based on the provided text, design and development validation should be performed according to planned and documented arrangements to ensure the resulting product meets requirements for its intended use.  The organization must document validation plans, including methods, acceptance criteria, and (where appropriate) statistical techniques with rationale for sample size.  Validation should be conducted on representative products (initial production units, batches, or equivalents), and the rationale for product selection should be recorded.  Validation must be completed before the product is released to the customer.  Results and conclusions, along with any necessary actions, must be documented.  If the device interfaces with other medical devices, validation must confirm that requirements are met when connected.



Final Conclusions of the Model

- Allows uploading a .pdf file
- The .pdf texts were divided into chunks suitable for the database
- The chunks are loaded into the ChromaDB vector database
- Understands the general context of a question
- Able to respond based on the context and the question
- The model answers simple questions such as: How to develop a design validation for medical devices


Improvements
- Improve the function to better clean up symbols and special characters that we are not interested in.
- Improve the division of the chunks. Where the cuts are being made
- The answer is not especially accurate
- It is not able to answer complex questions sucha as: 